<h2>Import necesary modules for analysis

In [1]:
import pandas as pd
import numpy as np
import requests
import geocoder
from os import environ

<h2>Gets the website and does a quick print to confirm process completed correctly

In [2]:
url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=946126446"

wiki = requests.get(url)

tables = pd.read_html(wiki.text)
print(len(tables))
tables

3


[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 282      M8Z         Etobicoke              Mimico NW
 283      M8Z         Etobicoke     The Queensway West
 284      M8Z         Etobicoke  Royal York South West
 285      M8Z         Etobicoke         South of Bloor
 286      M9Z      Not assigned           Not assigned
 
 [287 rows x 3 columns],
                                                   0   \
 0                                                NaN   
 1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
 2                                                 NL   
 3                                                  A   
 
                          

<h2>Reads the table into a dataframe

In [3]:
df_tor=pd.DataFrame(tables[0])
df_tor

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


 <h2>Only process the cells that have an assigned borough. Ignore cells with a borough that is "Not assigned".

In [4]:
df_tor.drop(df_tor[df_tor['Borough']=="Not assigned"].index,axis=0, inplace=True)
df_tor.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


<h2> More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [5]:
df_tor1=df_tor.groupby("Postcode").agg(lambda x:','.join(set(x)))
df_tor1.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


<h2> If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough. 

In [6]:
df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Neighbourhood']=df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Borough']
df_tor1.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


<h2>Resets the index

In [7]:
df_tor1 = df_tor1.reset_index()
df_tor1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2>Returns the shape of the table

In [8]:
df_tor1.shape

(103, 3)

<h2><b>Part 2

<h2>Adds the Latitude and Longitude columns

In [9]:
df_tor1['Latitude'] = ""
df_tor1['Longitude'] = ""
df_tor1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",,
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",,
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


<h2>Retrieves Google API key from environment variable

In [10]:
google_api = environ.get("googleapikey")

<h2>Uses the Google API to populate the dataframe with the latitude and longitude of the postal codes.

In [11]:
for i, postal_code in enumerate(df_tor1['Postcode']):
    # initialize your variable to None
    lat_lng_coords = None
    
    print(postal_code)
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code),key=google_api)
        print(g)
        print(g.latlng)
        lat_lng_coords = g.latlng        
    
    df_tor1['Latitude'][i] = lat_lng_coords[0]
    df_tor1['Longitude'][i] = lat_lng_coords[1]
    
df_tor1.head()

M1B
<[OK] Google - Geocode [Scarborough, ON M1B, Canada]>
[43.8066863, -79.1943534]
M1C
<[OK] Google - Geocode [Scarborough, ON M1C, Canada]>
[43.78453510000001, -79.1604971]
M1E
<[OK] Google - Geocode [Scarborough, ON M1E, Canada]>
[43.7635726, -79.1887115]
M1G
<[OK] Google - Geocode [Scarborough, ON M1G, Canada]>
[43.7709921, -79.2169174]
M1H
<[OK] Google - Geocode [Scarborough, ON M1H, Canada]>
[43.773136, -79.23947609999999]
M1J
<[OK] Google - Geocode [Scarborough, ON M1J, Canada]>
[43.7447342, -79.23947609999999]
M1K
<[OK] Google - Geocode [Scarborough, ON M1K, Canada]>
[43.7279292, -79.2620294]
M1L
<[OK] Google - Geocode [North York, ON M1L, Canada]>
[43.7111117, -79.2845772]
M1M
<[OK] Google - Geocode [Scarborough, ON M1M, Canada]>
[43.716316, -79.23947609999999]
M1N
<[OK] Google - Geocode [Scarborough, ON M1N, Canada]>
[43.692657, -79.2648481]
M1P
<[OK] Google - Geocode [Scarborough, ON M1P, Canada]>
[43.7574096, -79.273304]
M1R
<[OK] Google - Geocode [North York, ON M1R, Canad

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395


<h2>Final display of the dataframe

In [12]:
df_tor1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.8067,-79.1944
1,M1C,Scarborough,"Rouge Hill,Port Union,Highland Creek",43.7845,-79.1605
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
